<a href="https://colab.research.google.com/github/huune2k/BTVN_AI_8-8-2022/blob/AI_8-5-2022/FINAL_PROJECT_Bird_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !unzip "/content/gdrive/MyDrive/Colab/Bird1/Bird1-20220620T002605Z-001.zip" -d "/content/gdrive/MyDrive/Colab/Bird1"

In [4]:
import tensorflow as tf
from pathlib import Path

direction = Path('/content/gdrive/MyDrive/Colab/Bird1/Bird1')

In [5]:
# create dataset from file list 
filelist_ds = tf.data.Dataset.list_files(str(direction/'*/*'))

ds_size = 11788
train_size = int(ds_size*0.8)
val_size = int(ds_size*0.2)

filelist_ds = filelist_ds.shuffle(ds_size)
train_file = filelist_ds.take(train_size) 
val_file = filelist_ds.take(val_size) 

for file in filelist_ds.take(3):
  print(file)

tf.Tensor(b'/content/gdrive/MyDrive/Colab/Bird1/Bird1/156.White_eyed_Vireo/White_Eyed_Vireo_0132_158908.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/gdrive/MyDrive/Colab/Bird1/Bird1/082.Ringed_Kingfisher/Ringed_Kingfisher_0057_72812.jpg', shape=(), dtype=string)
tf.Tensor(b'/content/gdrive/MyDrive/Colab/Bird1/Bird1/088.Western_Meadowlark/Western_Meadowlark_0021_78841.jpg', shape=(), dtype=string)


In [6]:
def get_label(file_path):
  labels = []
  parts = tf.strings.split(file_path,'/')
  text = parts[-2]
  text_split = tf.strings.split(text,'.')
  number_label = int(text_split[0])
  labels = [1 if x == number_label else 0 for x in range(1,201,1)]
  return tf.convert_to_tensor(labels) 


In [7]:
img_h = 224
img_w = 224

def preprocessing(file_path):
  # read file
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3)

  # convert from uint8 to float32 and normalize values to [0, 1]
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize image
  img = tf.image.resize(img, [img_w, img_h])

  # get image label
  label = get_label(file_path)

  # return
  return img, label 

In [8]:
import numpy as np

batch_size = 16

# Build train dataset
train_ds = filelist_ds.shuffle(train_size)
train_ds = train_ds.map(preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
#train_ds = train_ds.map(lambda x:tf.function(preprocessing, [x], [tf.float32,tf.int32]))
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(1)

# Build val dataset
val_ds = filelist_ds.shuffle(val_size)
val_ds = val_ds.map(preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
#val_ds = val_ds.map(lambda x:tf.function(preprocessing, [x], [tf.float32,tf.int32]))
val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(1)
''' 
for file in train_ds.take(3):
  print(file)  '''

print(train_ds)
print(val_ds)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 200), dtype=tf.int32, name=None))>
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 200), dtype=tf.int32, name=None))>


In [9]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(224,224,3)))
model.add(layers.Conv2D(32, 3, (2,2), padding= 'valid', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(1,1), strides=2))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(32, 3, (2,2), padding= 'valid', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(1,1), strides=2))
model.add(layers.Dropout(0.25))

# model.add(layers.Conv2D(32, 3, (3,3), padding= 'valid', activation='relu'))
# model.add(layers.MaxPooling2D(pool_size=(1,1), strides=2))
# model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(200, activation='softmax'))


In [10]:
# compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

In [11]:
# train model
model.fit(train_ds, validation_data=val_ds, epochs=50)

Epoch 1/50
737/737 [==============================] - 1846s 2s/step - loss: 5.1775 - accuracy: 0.0120 - val_loss: 4.8779 - val_accuracy: 0.0378
Epoch 2/50
737/737 [==============================] - 53s 73ms/step - loss: 4.7250 - accuracy: 0.0447 - val_loss: 4.3626 - val_accuracy: 0.0934
Epoch 3/50
737/737 [==============================] - 53s 72ms/step - loss: 4.2797 - accuracy: 0.0843 - val_loss: 3.8031 - val_accuracy: 0.1906
Epoch 4/50
737/737 [==============================] - 53s 72ms/step - loss: 3.7018 - accuracy: 0.1720 - val_loss: 2.8515 - val_accuracy: 0.4159
Epoch 5/50
737/737 [==============================] - 54s 73ms/step - loss: 2.9739 - accuracy: 0.3092 - val_loss: 1.8981 - val_accuracy: 0.6216
Epoch 6/50
737/737 [==============================] - 54s 73ms/step - loss: 2.1296 - accuracy: 0.4927 - val_loss: 1.0935 - val_accuracy: 0.7950
Epoch 7/50
737/737 [==============================] - 53s 72ms/step - loss: 1.4414 - accuracy: 0.6445 - val_loss: 0.6243 - val_accuracy:

In [12]:
import pickle
file_name = "/content/gdrive/MyDrive/Colab/Bird1/birdtraining16batch50epoch.sav"
pickle.dump(model, open(file_name, 'wb'))

INFO:tensorflow:Assets written to: ram://5f072046-de69-421f-8607-3a5867beed58/assets
